### SparQL

### Linking Items in Wiki

In [8]:
import pandas as pd
import json
import requests
from SPARQLWrapper import SPARQLWrapper, JSON,POST
import time
from tqdm import tqdm

In [2]:
# 259 Modalities
json_file = 'updated_modalities.json' 
with open(json_file, 'r') as file:
    data = json.load(file)

modalities = data.get('updated modalities', [])
modality_df = pd.DataFrame(modalities, columns=['modality'])

modality_df['lower_modality'] = modality_df['modality'].str.lower() 
modality_df = modality_df.drop_duplicates(subset='lower_modality').drop(columns='lower_modality')

modality_df = modality_df.sort_values(by='modality').reset_index(drop=True)

print(modality_df)

          modality
0               2d
1         2d image
2     2d structure
3               3d
4    3d annotation
..             ...
254  visualization
255             vr
256            web
257  web structure
258       wireless

[259 rows x 1 columns]


In [3]:
# save to CSV
modality_df.to_csv('modality_259.csv', sep=',', index=True, header=True)

In [4]:
words = modality_df['modality'].tolist() 

In [5]:
# get QID from Wikidata Search API
def get_qid(word):
    url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={word}&language=en&format=json&limit=1"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['search']:
            return data['search'][0]['id']  # QID (e.g., 'Q42')
    return None

In [6]:
# Get QID from Wikidata Search API with exact matching
def get_qid_exact(word):
    url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={word}&language=en&format=json&limit=20"  # Get multiple results
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        if data['search']:
            for result in data['search']:
                # Check for exact match on the label
                if result['label'].lower() == word.lower():  # Case-insensitive comparison
                    return result['id']  # QID 
    return None

In [7]:
# Map words to QIDs
qid_mapping = {}

for word in tqdm(words, desc="Mapping words to QIDs"):
    qid = get_qid(word)
    if qid:
        qid_mapping[word] = qid
    else:
        qid_mapping[word] = None
    time.sleep(0.1)  # To avoid overloading the API

Mapping words to QIDs: 100%|██████████| 259/259 [01:37<00:00,  2.65it/s]


In [8]:
none_count = sum(1 for qid in qid_mapping.values() if qid is None)
total_count = len(qid_mapping)
none_percentage = (none_count / total_count) * 100

print(f"Words with None QID: {none_count}")
print(f"Percentage of None QID mappings: {none_percentage:.2f}%")

Words with None QID: 16
Percentage of None QID mappings: 6.18%


In [9]:
# Map exact words to QIDs
qid_mapping_exact = {}

for word in tqdm(words, desc="Mapping words to QIDs"):
    qid = get_qid_exact(word)
    if qid:
        qid_mapping_exact[word] = qid
    else:
        qid_mapping_exact[word] = None
    time.sleep(0.1)  # To avoid overloading the API

Mapping words to QIDs: 100%|██████████| 259/259 [01:43<00:00,  2.50it/s]


In [10]:
# exact word ratio
none_count = sum(1 for qid in qid_mapping_exact.values() if qid is None)
total_count = len(qid_mapping_exact)
none_percentage = (none_count / total_count) * 100

print(f"Words with None QID: {none_count}")
print(f"Percentage of None QID mappings: {none_percentage:.2f}%")

Words with None QID: 105
Percentage of None QID mappings: 40.54%


In [11]:
import csv

no_qid_words = {word: qid for word, qid in qid_mapping_exact.items() if qid is None}
has_qid_words = {word: qid for word, qid in qid_mapping_exact.items() if qid is not None}

with open('words_no_qid.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['word', 'qid'])  
    for word, qid in no_qid_words.items():
        writer.writerow([word, qid])

with open('words_with_qid.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['word', 'qid'])  
    for word, qid in has_qid_words.items():
        writer.writerow([word, qid])

In [13]:
qid_mapping_exact

{'2d': 'Q5651824',
 '2d image': None,
 '2d structure': None,
 '3d': 'Q108276326',
 '3d annotation': None,
 '3d body skeleton': None,
 '3d coordinate': None,
 '3d environment': None,
 '3d landmark': None,
 '3d layout': None,
 '3d lidar': None,
 '3d mesh': None,
 '3d model': 'Q3859833',
 '3d model parameter': None,
 '3d motion': None,
 '3d motion capture': None,
 '3d object': None,
 '3d point': None,
 '3d point cloud': None,
 '3d pose': None,
 '3d pose annotation': None,
 '3d scalar field': None,
 '3d scan': None,
 '3d simulation': None,
 '3d skeletal': None,
 '3d skeleton': None,
 '3d structure': None,
 '3d trajectory': None,
 '4d': 'Q20685814',
 'action': 'Q2634111',
 'activity': 'Q1914636',
 'algorithm': 'Q8366',
 'alpha matte': None,
 'analytic': 'Q65311416',
 'anatomical': None,
 'animation': 'Q11425',
 'annotation': 'Q857525',
 'application': 'Q166142',
 'audio': 'Q3500685',
 'audiovisual': 'Q758901',
 'behavioral': None,
 'binary': 'Q4913864',
 'bioassay data': None,
 'biochemical

In [14]:
none_qid_entries = {word: qid for word, qid in qid_mapping.items() if qid is None}
for word, qid in none_qid_entries.items():
    print(f"'{word}': {qid}")

'3d annotation': None
'3d body skeleton': None
'3d model parameter': None
'3d pose annotation': None
'3d scalar field': None
'control stream': None
'external tool': None
'network capture': None
'pose information': None
'positional data': None
'procedurally generated data': None
'robot joint angle': None
'spiking neural data': None
'text-image interaction': None
'user metadata': None
'virtual api': None


In [15]:
qid_entries = {word: qid for word, qid in qid_mapping.items() if qid is not None}
for word, qid in qid_entries.items():
    print(f"'{word}': {qid}")

'2d': Q2815842
'2d image': Q51593562
'2d structure': Q51941403
'3d': Q189177
'3d coordinate': Q129926772
'3d environment': Q54457512
'3d landmark': Q38215966
'3d layout': Q128229676
'3d lidar': Q119704469
'3d mesh': Q37115292
'3d model': Q3859833
'3d motion': Q46043750
'3d motion capture': Q104559750
'3d object': Q35459920
'3d point': Q128304176
'3d point cloud': Q128304176
'3d pose': Q4636322
'3d scan': Q94701573
'3d simulation': Q45045
'3d skeletal': Q37647591
'3d skeleton': Q49885563
'3d structure': Q105696529
'3d trajectory': Q92923419
'4d': Q412616
'action': Q188473
'activity': Q1656682
'algorithm': Q8366
'alpha matte': Q57590135
'analytic': Q485223
'anatomical': Q2725967
'animation': Q202866
'annotation': Q857525
'application': Q166142
'audio': Q482994
'audiovisual': Q758901
'behavioral': Q846566
'binary': Q50053
'bioassay data': Q97060518
'biochemical': Q7094
'biological': Q7094
'biomarker': Q864574
'biometric': Q177765
'biosample': Q96797376
'biosignal': Q644240
'blockchain': Q

In [16]:
none_qid_entries = {word: qid for word, qid in qid_mapping_exact.items() if qid is None}
for word, qid in none_qid_entries.items():
    print(f"'{word}': {qid}")

'2d image': None
'2d structure': None
'3d annotation': None
'3d body skeleton': None
'3d coordinate': None
'3d environment': None
'3d landmark': None
'3d layout': None
'3d lidar': None
'3d mesh': None
'3d model parameter': None
'3d motion': None
'3d motion capture': None
'3d object': None
'3d point': None
'3d point cloud': None
'3d pose': None
'3d pose annotation': None
'3d scalar field': None
'3d scan': None
'3d simulation': None
'3d skeletal': None
'3d skeleton': None
'3d structure': None
'3d trajectory': None
'alpha matte': None
'anatomical': None
'behavioral': None
'bioassay data': None
'biological': None
'biometric': None
'body motion': None
'bounding box': None
'brain signal': None
'camera parameter': None
'camera pose': None
'categorical': None
'chemical': None
'control stream': None
'cwl': None
'data array': None
'demographic': None
'dynamical': None
'ecg': None
'edge map': None
'electrophysiological': None
'eog': None
'epigenetic': None
'evaluation metric': None
'external tool

In [ ]:
# Load the words_with_qid.csv file and create a mapping
word_to_qid = {}
with open('words_with_qid.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        word_to_qid[row['word']] = row['qid']


In [22]:
!pip3 install openpyxl


In [23]:
# Load the words_no_qid_update.xlsx file
words_no_qid_update = pd.read_excel('words_no_qid_update.xlsx')

# Replace the 'word' column with 'wikidata' column
words_no_qid_update['word'] = words_no_qid_update['wiki_word']

# Drop the 'wikidata' column
words_no_qid_update = words_no_qid_update.drop(columns=['wiki_word'])

In [25]:
# Convert the DataFrame to a dictionary
words_no_qid_update_dict = words_no_qid_update.set_index('word').to_dict()['qid']

# Combine the dictionaries and remove duplicates
combined_data = {**word_to_qid, **words_no_qid_update_dict}

# Save the combined data to a new CSV file
with open('combined_words_with_qid.csv', 'w', encoding='utf-8', newline='') as csvfile:
    fieldnames = ['word', 'qid']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for word, qid in combined_data.items():
        writer.writerow({'word': word, 'qid': qid})

print("Combined data has been saved to combined_words_with_qid.csv.")

Combined data has been saved to combined_words_with_qid.csv.


In [11]:
#sort the combined data
combined_data_df = pd.read_csv('combined_words_with_qid.csv',index_col=0)
combined_data_df = combined_data_df.sort_values(by='word').reset_index(drop=True)
# Save the sorted data to a new CSV file
combined_data_df.to_csv('combined_words_with_qid.csv', sep=',', index=False, header=True)

In [9]:
combined_data_df = pd.read_csv('combined_words_with_qid.csv')
all_qids = combined_data_df['qid'].dropna().unique().tolist()

print(len(all_qids))

batch_size = 50  
chunk_size = 150 

# Split the list into chunks
def split_chunks(lst, size):
    return [lst[i:i+size] for i in range(0, len(lst), size)]

batches = split_chunks(all_qids, batch_size)  
all_qids_chunks = split_chunks(all_qids, chunk_size)  



226


### Typical properties to link one item to another:

**Comparing items ...**  
said to be the same as (P460)  
different from (P1889) - (different from, but sometimes confused with ...)

**Item is part of ...**  
part of (P361) - (section of .../ contained within .../ pieces of ...)  
instance of (P31) - (is an example of ...)  
subclass of (P279) - (is a subset of ...)  
facet of (P1269) - (aspect of .../ subitem of .../ a broader perspective on the same topic is offered by ...)  
has characteristic (P1552) - (has characteristic .../ defining feature .../ inherent property ...)

**Item contains ...**  
has part(s) (P527) - (contains ...)  
has part(s) of the class (P2670) (has parts that are instances of .../ some parts form subclass of ...)

**Others**  
related property (P1659) (used to indicate another property that might provide additional information about the subject)

file format (P2701)


### Two-end

In [10]:
import random
# Disable SSL verification
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [11]:
# 初始化SPARQL
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setMethod(POST) 
sparql.setReturnFormat(JSON)
sparql.addCustomHttpHeader("User-Agent", "MyWikidataBot/1.0 (yu.wang3@uzh.ch)")

def query_relations(qids_batch, word2_chunk, max_retries=5):
    """查询当前批次QID与某一块word2_chunk的关系"""
    qids_values = " ".join([f"wd:{qid}" for qid in qids_batch])
    word2_values = " ".join([f"wd:{qid}" for qid in word2_chunk])
    
    query = f"""
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    
    SELECT ?word ?wordLabel ?relation ?relationLabel ?word2 ?word2Label
    WHERE {{
      VALUES ?word {{ {qids_values} }}
      VALUES ?word2 {{ {word2_values} }}
      ?word ?relation ?word2 .
      FILTER (?relation IN (wdt:P460, wdt:P1889, wdt:P361, wdt:P31, wdt:P279, wdt:P1269, wdt:P1552, wdt:P527, wdt:P2670, wdt:P1659, wdt:P2701, wdt:P1382))
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    retries = 0
    while retries < max_retries:
        try:
            sparql.setQuery(query)
            results = sparql.query().convert()
            return results["results"]["bindings"]
        except Exception as e:
            if "429" in str(e):
                wait = random.uniform(10, 30)
                print(f"429 error，waiting{wait:.1f}seconds and retry...")
                time.sleep(wait)
                retries += 1
            else:
                print(f"error: {e}")
                return []
    print("Max retries reached.")
    return []

In [12]:
# query relations
all_results_1 = []
for batch in tqdm(batches, desc="Processing"):
    for chunk in all_qids_chunks:
        results = query_relations(batch, chunk)
        all_results_1.extend(results)
        time.sleep(random.uniform(1, 3))  # 短等待避免频繁429
    time.sleep(random.uniform(5, 10))     # 批次间较长等待

print(f"Totally find {len(all_results_1)} relations。")

Processing: 100%|██████████| 5/5 [00:57<00:00, 11.52s/it]

Totally find 53 relations。


In [13]:
# Store results
def store_results_1(all_results):
    output = set()
    
    for result in all_results:
        word = result['word']['value'].split('/')[-1] 
        word_label = result.get('wordLabel', {}).get('value', "")  
        relation = result['relation']['value'].split('/')[-1]
        #relation_label = result.get('relationLabel', {}).get('value', "")  
        word2 = result['word2']['value'].split('/')[-1] 
        word2_label = result.get('word2Label', {}).get('value', "")  

        # **去重**
        triple = (word, word_label, relation, word2, word2_label)
        output.add(triple)

    return [dict(zip(["word", "word_label", "relation", "word2", "word2_label"], row)) for row in output]

In [14]:
# Convert to DataFrame and save to CSV
output_df = pd.DataFrame(store_results_1(all_results_1))
output_df.to_csv('modality_relations_two_end.csv', index=False)

### One-end

In [42]:
# Query the SPARQL endpoint for each batch
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setReturnFormat(JSON)

def query_relations_ood(qids_batch):
    # Create SPARQL query for the batch
    qids_values = " ".join([f"wd:{qid}" for qid in qids_batch])
    
    query = f"""
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    
    SELECT ?word ?wordLabel ?relation ?relationLabel ?relatedWord ?relatedWordLabel
    WHERE {{
      VALUES ?word {{ {qids_values} }}
      # VALUES ?relatedWord {{ {qids_values} }}
      ?word ?relation ?relatedWord .
      FILTER (?relation IN (wdt:P460, wdt:P1889, wdt:P361, wdt:P31, wdt:P279, wdt:P1269, wdt:P1552, wdt:P527, wdt:P2670, wdt:P1659)) .
      
      # Retrieve labels for word, relation, and relatedWord
      SERVICE wikibase:label {{
        bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
        ?word rdfs:label ?wordLabel .
        # ?relation rdfs:label ?relationLabel .
        ?relatedWord rdfs:label ?relatedWordLabel .
      }}
    }}
    """
    sparql.setQuery(query)
    results = sparql.query().convert()
    return results["results"]["bindings"]

In [43]:
# Disable SSL verification
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [47]:
# Process each batch
# two-end
all_results = []
for i, batch in enumerate(tqdm(batches, desc="Processing batches")):
    qids_batch = batch["qid"].dropna().tolist()
    batch_results = query_relations(qids_batch)
    #print(batch_results)
    all_results.extend(batch_results)
    time.sleep(5)  # To avoid overloading the SPARQL endpoint

Processing batches: 100%|██████████| 5/5 [00:26<00:00,  5.33s/it]


In [48]:
all_results

[{'word': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q514'},
  'relatedWord': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q420'},
  'relation': {'type': 'uri',
   'value': 'http://www.wikidata.org/prop/direct/P279'},
  'wordLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'anatomy'},
  'relatedWordLabel': {'xml:lang': 'en',
   'type': 'literal',
   'value': 'biology'}},
 {'word': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q514'},
  'relatedWord': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q420'},
  'relation': {'type': 'uri',
   'value': 'http://www.wikidata.org/prop/direct/P361'},
  'wordLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'anatomy'},
  'relatedWordLabel': {'xml:lang': 'en',
   'type': 'literal',
   'value': 'biology'}},
 {'word': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q864574'},
  'relatedWord': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q420'},
  'relation': {'typ

In [53]:
# Process each batch
# one-end
all_results_ood = []
for i, batch in enumerate(tqdm(batches, desc="Processing batches")):
    qids_batch = batch["qid"].dropna().tolist()
    batch_results = query_relations_ood(qids_batch)
    #print(batch_results)
    all_results_ood.extend(batch_results)
    time.sleep(3)  # To avoid overloading the SPARQL endpoint

Processing batches: 100%|██████████| 5/5 [00:22<00:00,  4.59s/it]


In [54]:
all_results_ood

[{'word': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q420'},
  'relation': {'type': 'uri',
   'value': 'http://www.wikidata.org/prop/direct/P31'},
  'relatedWord': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q2465832'},
  'wordLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'biology'},
  'relatedWordLabel': {'xml:lang': 'en',
   'type': 'literal',
   'value': 'branch of science'}},
 {'word': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q420'},
  'relation': {'type': 'uri',
   'value': 'http://www.wikidata.org/prop/direct/P31'},
  'relatedWord': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q4671286'},
  'wordLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'biology'},
  'relatedWordLabel': {'xml:lang': 'en',
   'type': 'literal',
   'value': 'academic major'}},
 {'word': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q420'},
  'relation': {'type': 'uri',
   'value': 'http://www.wikidata.org/prop/direct/P31'}

In [49]:
# Store results
def store_results(all_results):
    output = []
    
    for result in all_results:
        word = result['word']['value'].split('/')[-1] 
        word_label = result.get('wordLabel', {}).get('value', "")  
        relation = result['relation']['value'].split('/')[-1]
        #relation_label = result.get('relationLabel', {}).get('value', "")  
        related_word = result['relatedWord']['value'].split('/')[-1] 
        related_word_label = result.get('relatedWordLabel', {}).get('value', "")  

        output.append({
            'word': word,
            'word_label': word_label,
            'relation': relation,
            #'relation_label': relation_label, 
            'related_word': related_word,
            'related_word_label': related_word_label
        })
    
    return output

In [50]:
store_results(all_results)

[{'word': 'Q514',
  'word_label': 'anatomy',
  'relation': 'P279',
  'related_word': 'Q420',
  'related_word_label': 'biology'},
 {'word': 'Q514',
  'word_label': 'anatomy',
  'relation': 'P361',
  'related_word': 'Q420',
  'related_word_label': 'biology'},
 {'word': 'Q864574',
  'word_label': 'biomarker',
  'relation': 'P361',
  'related_word': 'Q420',
  'related_word_label': 'biology'},
 {'word': 'Q420',
  'word_label': 'biology',
  'relation': 'P527',
  'related_word': 'Q514',
  'related_word_label': 'anatomy'},
 {'word': 'Q20514253',
  'word_label': 'blockchain',
  'relation': 'P279',
  'related_word': 'Q8513',
  'related_word_label': 'database'},
 {'word': 'Q8513',
  'word_label': 'database',
  'relation': 'P527',
  'related_word': 'Q42848',
  'related_word_label': 'data'},
 {'word': 'Q1172540',
  'word_label': 'data stream',
  'relation': 'P527',
  'related_word': 'Q42848',
  'related_word_label': 'data'},
 {'word': 'Q1643805',
  'word_label': 'container',
  'relation': 'P279',
 

In [52]:
# Convert to DataFrame and save to CSV
output_df = pd.DataFrame(store_results(all_results))
output_df.to_csv('modality_relations_two_end.csv', index=False)

In [55]:
store_results(all_results_ood)

[{'word': 'Q420',
  'word_label': 'biology',
  'relation': 'P31',
  'related_word': 'Q2465832',
  'related_word_label': 'branch of science'},
 {'word': 'Q420',
  'word_label': 'biology',
  'relation': 'P31',
  'related_word': 'Q4671286',
  'related_word_label': 'academic major'},
 {'word': 'Q420',
  'word_label': 'biology',
  'relation': 'P31',
  'related_word': 'Q11862829',
  'related_word_label': 'academic discipline'},
 {'word': 'Q420',
  'word_label': 'biology',
  'relation': 'P279',
  'related_word': 'Q7991',
  'related_word_label': 'natural science'},
 {'word': 'Q420',
  'word_label': 'biology',
  'relation': 'P460',
  'related_word': 'Q864928',
  'related_word_label': 'life sciences'},
 {'word': 'Q420',
  'word_label': 'biology',
  'relation': 'P527',
  'related_word': 'Q431',
  'related_word_label': 'zoology'},
 {'word': 'Q420',
  'word_label': 'biology',
  'relation': 'P527',
  'related_word': 'Q441',
  'related_word_label': 'botany'},
 {'word': 'Q420',
  'word_label': 'biolog

In [56]:
# Convert to DataFrame and save to CSV
output_df = pd.DataFrame(store_results(all_results_ood))
output_df.to_csv('modality_relations_one_end.csv', index=False)